## langchain.chains.APIChain

This appears to be the most basic langchain API-calling feature. The API endpoint is given in plain text (in a semi-tabular format), and there doesn't appear to be any planning for using multiple endpoints if required.

In [1]:
from langchain.chains import APIChain
from langchain.chat_models import ChatOpenAI

import pprint
import dotenv

dotenv.load_dotenv()

pp = pprint.PrettyPrinter(indent=4)

Load a model, load the docs and create the chain. In this utility, we give the documentation in a semi-tabular form. See https://python.langchain.com/en/latest/modules/chains/examples/api.html?highlight=APIChain.

In [2]:
llm = ChatOpenAI(model = "gpt-4", temperature=0)

with open("monarch_wrapper_docs.txt") as f:
    docs = f.read()
    
print(docs)

chain_new = APIChain.from_llm_and_api_docs(llm, docs, verbose=True)

API documentation:
Endpoint: http://localhost:3434

GET /search

This endpoint is for searching for biomedical entities such as Diseases, Genes, and Phenotypes, by their common name.

Query parameters table:
term | string | Search term, e.g., "Marfan's Syndrome", "CYP6B", "Microcephaly" | required
category | string | What category to search in? Available values: biolink:Disease, biolink:Gene, biolink:Phenotype | required
limit | integer | Maximum number of results to return. A valid value should be an integer between 1 and 10 (inclusive) | required
offset | integer | Number of results to skip (for pagination) | required

Response schema (JSON object):
results | array[object] | Array of Match Result Objects
total | integer | Total number of matches in the database

Each object in the "results" key has the following schema:
id | string | The CURIE identifier for the match
name | string | The human-readable name of the match
categories | string | A list of categories the match belongs to


Run a query

In [3]:
%%time
output = chain_new.run("What phenotypes are associated with Cystic Fibrosis?")



> Entering new APIChain chain...
http://localhost:3434/search?term=Cystic%20Fibrosis&category=biolink:Disease&limit=1&offset=0
{"results":[{"id":"MONDO:0009061","name":"cystic fibrosis","categories":["biolink:Disease"],"description":"Cystic fibrosis (CF) is a genetic disorder characterized by the production of sweat with a high salt content and mucus secretions with an abnormal viscosity."}],"total":16}

> Finished chain.
CPU times: user 29.7 ms, sys: 8.79 ms, total: 38.4 ms
Wall time: 12.5 s


In [4]:
pp.pprint(output)

('Cystic Fibrosis (CF) is a genetic disorder characterized by the production '
 'of sweat with a high salt content and mucus secretions with an abnormal '
 'viscosity. The disease has the identifier MONDO:0009061.')
